In [5]:
# API support
import gspread # Google API
import facebook # Facebook API
from twython import Twython # Twitter API
from oauth2client.service_account import ServiceAccountCredentials # for google api

# features
import hashlib # Encryption for UserID
import re # Images + DM Filtering
import requests # Image download
import os # image deletion
import spacy # Names redaction
import sklearn as sk # Filteration

ModuleNotFoundError: No module named 'sklearn'

In [2]:
###
# FB Graph Explorer API
"""
FB_PAGE_ACCESS_TOKEN = ""
FB_PAGE_ID = ""
graph = facebook.GraphAPI(PAGE_ACCESS_TOKEN)
"""
###
# Twitter
TW_CONSUMER_API = ""
TW_CONSUMER_SECRET_API = ""
TW_ACCESS_TOKEN = ""
TW_ACCESS_TOKEN_SECRET = ""
twitter = Twython(TW_CONSUMER_API, TW_CONSUMER_SECRET_API, TW_ACCESS_TOKEN, TW_ACCESS_TOKEN_SECRET)
###
# Google Drive
scope =['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)
###

INITIAL_COUNTER = 0 # replace me. if you change the txt file, also replace this
                    # used to extract the row of the contact information from drive

In [3]:
def hashFunc(string):
    """
    Stable hash function
    """
    h = hashlib.blake2b(key=b'ucsdCnfssns', digest_size=10)
    h.update(string.encode('UTF-8'))
    return h.hexdigest();

def downloadImage(imageUrl, localFileName):
    """
    download image
    """
    response = requests.get(imageUrl)
    if response.status_code == 200:
        print('Downloading %s...' % (localFileName))
        with open(localFileName, 'wb') as fo:
            for chunk in response.iter_content(4096):
                fo.write(chunk)
    print("Complete Downloading " + localFileName)

def next_available_row(worksheet):
    """
    find the next empty row on google sheets
    """
    str_list = list(filter(None, worksheet.col_values(1)))  # fastest
    return str(len(str_list)+1)

def clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

def clean_str(s):
    """
    gets rid of punctuations
    """
    return s.lower().apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))

In [4]:
# Post Number Retrieval
with open("counter.txt", "r") as c:
    counter = int(c.readline())

In [5]:
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
toPost = client.open("UCSD_Confessions_Form").sheet1
posted = client.open("UCSD_Confessions_Form").worksheet("Posted")

toRemove = client.open("UCSD Confessions Moderation Form").sheet1
postsRemoved = client.open("UCSD Confessions Moderation Form").worksheet("Removed")
notRemove = client.open("UCSD Confessions Moderation Form").worksheet("Not Removed")

In [6]:
# Extract and print all of the values
listOfPosts = toMake.get_all_values()
pRemoval = toRemove.get_all_values()

In [ ]:
# filteration model
m = sk.externals.joblib.load('bnb.joblib') 

In [8]:
# Get the posts
# 0: Timestamp, 1: confession, 2: tag, 3: Contact Info, 4: Images, 5: ID Code
for row in listOfPosts[1:]: # TODO: Reverse the iteration so its a stack
    counter += 1
    # String building
    post = f"#{counter}\n" # Number system
    post += f"[{row[2].strip()}]\n" # Tag
    if (bool(row[5].strip())):
        post += f"User ID: {hashFunc(row[5].strip())}\n" # Hash of ID
    if (bool(row[3].strip())):
        post += f"Poster has provided contact information\n" # Contact info
    post += f"{row[1].strip()}" # Confession post
    
    # Insert Filtration System here
    # Bayesian spam filter for off topic things
    # NER for name redaction
    """
    # spam filter
    if m.model.predict_proba(clean_str(post)) > .99: # 1 = Bad post, 0 = good post
        posted.resize(next_available_row(posted))
        posted.append_row([counter] + row + [False])
        #postToMake.delete_row(rowNumber) # TODO: for loop reversal in order to fix this
        continue
    """
    
    # Edge Case 1: Post is greater than 280
    if (len(post) > 280):
        numOfPostsRequired = (len(post) - 270) // 260 + 2
        # With image
        if re.match(r"^https:\/\/i\.imgur\.com\/[a-z0-9A-Z]{7}\.jpg$", f"{row[4].strip()}"):
            initial_p = post[0:270]
            initial_p += f"\n1/{numOfPostsRequired}"
            
            downloadImage(row[4].strip(), "images/" + str(counter) + ".jpg")
            photo = open("images/" + str(counter) + ".jpg", 'rb')

            response = twitter.upload_media(media=photo)
            postID = twitter.update_status(status=initial_p, media_ids=[response['media_id']])["id_str"]
            for index, i in enumerate(range(270, len(post), 260)):
                p = "@UCSDConfession\n"
                p += post[i: i + 260]
                p += f"\n{index + 2}/{numOfPostsRequired}"
                postID = twitter.update_status(status=p, in_reply_to_status_id=postID)["id_str"]
            photo.close()
        # Without image
        else:
            p = post[0: 270]
            p += f"\n1/{numOfPostsRequired}"
            postID = twitter.update_status(status=p)["id_str"]
            for index, i in enumerate(range(270, len(post), 260)):
                p = post[i: i + 260]
                p += f"\n{index + 2}/{numOfPostsRequired}"
                postID = twitter.update_status(status=p, in_reply_to_status_id=postID)["id_str"]
    # Edge Case 2: Images w/ short confession
    elif re.match(r"^https:\/\/i\.imgur\.com\/[a-z0-9A-Z]{7}\.jpg$", f"{row[4].strip()}"):
        downloadImage(row[4].strip(), "images/" + str(counter) + ".jpg")
        photo = open("images/" + str(counter) + ".jpg", 'rb')
        response = twitter.upload_media(media=photo)
        twitter.update_status(status=post, media_ids=[response['media_id']])
        os.remove("images/" + str(counter) + ".jpg")
        photo.close()
    # Default Case - short confession
    else:
        twitter.update_status(status=post) 
    """
    # For FB posts
    graph.put_object(parent_object=PAGE_ID, connection_name='feed',
                      message=post)
    if row[4] == "":
        graph.put_object(parent_object=PAGE_ID, connection_name='feed',
                      message=post)
    else:
        # for images
        link = row[5]
        downloadImage(link, "images/" + str(counter) + ".png")
        graph.put_photo(parent_object=PAGE_ID, connection_name='feed', image=open('images/' + str(counter) + '.png', 'rb'),
                message=post)
    """
    # Transfer posts to posted
    posted.resize(next_available_row(posted))
    posted.append_row([counter] + row + [True]) # Make sure to modify to see if it passes filter
    #postToMake.delete_row(rowNumber)

In [9]:
VERIFIED_MOD_CODES = [""]
# 0: Timestamp, 1: Personal Code, 2: Post ID, 3: Reason for Deletion, 4: Post Number
for row in pRemoval[1:]: # reversal
    if row[1].strip() not in VERIFIED_MOD_CODES:
        notRemoved.resize(next_available_row(postsNotRemoved))
        notRemoved.append_row(row)
    else:
        postsRemoved.resize(next_available_row(postsRemoved))
        postsRemoved.append_row(row)
        twitter.destroy_status(id=row[2].strip()) # doesn't delete long chained tweets
# Delete rows
for row in reversed(range(2, len(postToMake.col_values(1)) + 1)):
    toMake.delete_row(row)


In [11]:
# Delete rows
try:
    for row in reversed(range(2, len(pRemoval.col_values(1)) + 1)):
        pRemoval.delete_row(row)
        # len(pRemoval.col_values(1)) + 1 - i???
except AttributeError:
    print("No Values")

No Values


In [12]:
# Save post number
with open("counter.txt", "w") as c:
    c.write(str(counter))

In [ ]:
###
"""
# Automating the replys for people looking for contact information
DMs = graph.get_object(PAGE_ID + "/conversations", fields="snippet")["data"]
messages = []
for dm in DMs:
    if re.match(r"^CONTACT_INFO: #\d+$", dm["snippet"]):
        index = int(re.findall('^CONTACT_INFO: #(\d+)$',dm["snippet"])[0])
        try:
            row = posted.row_values(index - INITIAL_COUNTER)
            m = f"Contact Information for post #{index}: " + posted.row_values(index)[4]
            graph.put_object(parent_object=dm["id"], connection_name="messages", message=m)
        except:
            m = f"Contact Information for post #{index} is invalid. Please make sure you input the correct value"
            graph.put_object(parent_object=dm["id"], connection_name="messages", message=m)
    elif re.match(r"^CONTACT_INFO: [(#\d+,?)*#\d+]$", dm["snippet"]):
        indexesString = re.match(r"^CONTACT_INFO: ([(#\d+,?)*#\d+])$", dm["snippet"])
        indexes = indexesString.strip('][').split(',')
        for i in indexes:
            stringIndex = i.strip(" ").strip("#")
            try:
                index = int(stringIndex)
                row = posted.row_values(index)
                messages.append(f"Contact Information for post #{index}: " + posted.row_values(index)[4])
                graph.put_object(parent_object=dm["id"], connection_name="messages", message=m)
            except ValueError:
                messages.append(f"#{stringIndex} is not a valid post number")
            except APIError:
                messages.append(f"Contact Information for post #{index} does not exist. Please make sure you input the correct value")
        for m in messages:
            graph.put_object(parent_object=dm["id"], connection_name="messages", message=m)
"""
###
# TODO: Twitter replies. Look at twitter api.